# Merging Datasets with Baltimore City Open Data

In Excel, we used the `VLOOKUP` function to combine data in different workbooks or sheets based on shared ID column data in neighboring rows. To do this, we wrote the function: 
```
=VLOOKUP(cell in old dataset that matches a value in the first column of new dataset, 
        selected data table of new dataset, 
        row number of new data to add to old dataset, 
        exact match (0) OR approximate match (1))
```
which allowed us to add a column of data from *new dataset* to *old dataset* based on some shared/linking data. 

This is super helpful and time efficient if we want to combine or condense information from different data sources our outputs, however, we're limited in the number of data columns we can transfer at a time (one) and how our final dataset appears (for example, if we want to only keep data that's in both sheets, we need to add an extra step of filtering out the null cells in the post-VLOOKUP spreadsheet. We can use **pandas merge** function to combine larger datasets in a similar way in Python, but with broader functionality.

## [Pandas Merge](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.merge.html) 

Pandas merge allows us to merge two pandas dataframes together using the following convention: 

```
merged_df = pd.merge(left_df, 
                    right_df, 
                    how='inner', 
                    on=None, 
                    left_on=None, 
                    right_on=None, 
                    left_index=False, 
                    right_index=False, 
                    sort=False, 
                    suffixes=('_x', '_y'), 
                    copy=True, 
                    indicator=False, 
                    validate=None)
```
where we'll mostly need to be concerned with: 

 - __merged_df__ is the newly defined dataframe from the merged data
 - __pd__ is because we're using a pandas function
 - __merge__ is the pandas function that we're using to merge the dataframes
 - __left_df__ is the name of the dataframe that's our designated "left" dataframe
 - __right_df__ is the name of the dataframe that's our designated "right" dataframee
 - __how__ tells our dataframe how we want to merge the data (more details below)
 - __left_on__ is the name of the column that contains our "matching data" in the left dataframe
 - __right_on__ is the name of the column that contains our "matching data" in the right dataframe
 - __on__ is what we can use to define the column that contains our "matching data" is the column name is the same in both dataframes

The rest of these values are the default merge functions (defined [here](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.merge.html)), most of which we won't often need to tweak.

### Dataframes to Merge

In Excel, we can only transer data from one dataset to another, but in Python, we can merge all of the information from both datasets together, if we want. To merge our data, we'll identify the left and right data frame based on what data we want to merge onto existing data. 

For example, if we want to only merge a few columns from dataset df_A onto dataset df_B, we'll want to define df_B as the left dataframe and df_A as the right dataframe because we want to keep all of the data from df_B and add on some columns from df_A. However, we we want to merge all of the columns from df_A and df_B, then it doesn't really matter which dataframe we define as the left and right dataframe.

### Types of Merging 

Unlike in Excel, we can merge our data in in many different variations through the merge parameter in the merge function depending on how we define our right and left dataframes. In general, we can merge our left and right dataframes in the following ways:
 - __"left"__ merges data from the right dataframe onto the left dataframe *only if* if has a key that matches values in the left dataframe and fills in "NaN" (null) values in rows that don't have a value in the key column from the right dataframe
 - __"right"__ merges data from the left dataframe onto the right dataframe *only if* if has a key that matches values in the right dataframe and fills in "NaN" (null) values in rows that don't have a value in the key column from the left dataframe
 - __"outer"__ merges together and all of the data in both dataframes on the key column and fills in an "NaN" value for the columns and rows that don't have values or matches in the key column
 - __"inner"__ merges together only data that's in *both* dataframes
 
To get a better idea of what this means, we'll run through some examples of dataframe mergine with [Baltimore City 911 Call data](https://data.baltimorecity.gov/Public-Safety/911-Police-Calls-for-Service/xviu-ezkt), [Baltimore Victim Crime Data](https://data.baltimorecity.gov/Public-Safety/BPD-Part-1-Victim-Based-Crime-Data/wsfq-mvij), and [Baltimore City Police Department Arrest Data](https://data.baltimorecity.gov/Public-Safety/BPD-Arrests/3i3v-ibrt).

## import packages 

In [80]:
# import packages for data analysis and visualization


## import data 

In [81]:
# import bpd arrest data


In [82]:
# preview arrest data


In [83]:
# import baltimore city 911 call data


In [84]:
# preview 911 data:


In [85]:
# import victim crime data


In [86]:
# preview crime data


In our datasets we have a lot of information about arrests, crime, and 911 calls in Baltimore. Let's say that we want to aggregate all of this to look at the number of 911 calls, arrests, and crimes happening every year. 

To do this, we'll first need to aggregate our dataframes to count the number of occurrances per year. 

## Data Cleaning 

In [87]:
# look at the data types for each of our datasets (911)


In [88]:
# df_arrest info

In [89]:
# df crime info


### Convert columns to datetime datatype 

We see that none of our datasets have our date or time values as datasets, so we'll need to convert them to datetime formats similar to our work last week.

In [90]:
# create a new column in the arrest data with both of the date and time data


In [91]:
# convert arrest date time column to a datetime data type


In [92]:
# create a new column in the crime dataset with both the date and time data


In [93]:
# convert crime date time column into a datetime datatype


We need to define the format of our datetime object in the 911 call data because we have an AM/PM string in the date time value. We also need to do this if our datetime string that we want to convert into a datetime object is in any format other than `month/date/year 24hour:min:second`

A full list of the formatting for datetime is listed [here](https://www.programiz.com/python-programming/datetime/strftime). Our 911 data's CallDateTime column lists values like `11/02/2016 04:46:00 PM`, so we'll need to define our datetime column with: 

 - __%m__ for month
 - __%d__ for day
 - __%Y__ for 4-digit year
 - __%I__ for 12-hour clock
 - __%M__ for minute
 - __%S__ for second
 - __%p__ for am/pm

In [94]:
# convert the 911 date time column into a datetime datatype (since the values are already together in one column)


Now if we look the `df.info()` all of the date time datatypes should be datetime datatypes.

#### Data Aggregation by Year

Now, let's aggregate the counts of each of our datatypes by the year by first creating a "year" column in each of our dataframes, then using the pandas groupby function to aggregate the counts of each dataset per year. 


We can either do this one at a time for each of our datasets similar to what we've done before, however, since we're using the dt.year function for all three datasets, we can take advantage of a python __for loop__ to do this with a little less code using the following convention: 
```
for __value__ in __range or list__:
    *do some action*
```
Which will allow us to perform the same action to whatever we've defined in the list. We want to add a "year" column to each of our dataframes by calling the same df.year function on the DateTime column, so we can make a list of the dataframes and tell our loop to create a new column based on the DateTime column for each dataframe. Note that this works since all of the date time columns are named the same "DateTime" name. If they weren't, we would need to rename the columns to make this for loop work.
  

In [95]:
# create list of dataframes


We'll write: 

```
for df in df_list:
    df["year"] = df["DateTime"].dt.year
```

Where the __df__ is a placeholder that we've defined as a variable for the df in our repeating action. This can be anything as long as it's consistent and the same in the functions within the for loop. __df_list__ is the list we defined with all of our dataframes.

Essentially what we're saying is that for each value in our df_lists, plug in each list value any time you see a "df" and perform the function. 

In [96]:
# write a for loop to add a year column to the data


In [97]:
# so if we preview the crime data now, we see the year column


Now, we can aggregate the counts by year. We'll do this individually for this round since all of the columns that we want to count are named different things. 

Since we're combining all of this data at the end, we'll also want to rename our column data to specify which counts are in that column by defining the column name in the `.agg` function

In [98]:
# aggregate counts of 911 data


In [99]:
# preview data


In [100]:
# aggregate counts of crime


In [101]:
# preview data


In [102]:
# aggregate counts of arrests


In [103]:
# preview data


### Merging Data 

Now we can finally merge all of our data into one dataframe that merges the counts of crime, 911, and arrest data into one dataframe.

If we look at the length of each dataframe, we can see that some of the datasets have data for more years than other datasets: 

In [104]:
# length of crime year counts


In [105]:
# length of 911 call year counts


In [106]:
# length of arrest year counts


Since the crime count has the most years listed, we'll merge the arrest and crime counts onto the crime count dataframe with the `df.merge` function: 

In [107]:
# merge the 911 call counts onto the crime count dataframe


In [108]:
# preview tail data


In [109]:
# merge the arrest count onto the merged df


In [110]:
# final dataset


Since we only have useable data for 2014-2019, we'll filter our merged dataframe to only include those rows    

In [111]:
# filter dataframe based on values in the year column



In [112]:
#preview data


Let's make a line plot to look at the changes of all of these over time. Since our data isn't formatted how we need it (all x values in one column and all y values in another column), we'll need to use the pandas `melt` function to rearrange our data in a format that works with plotly express.

In [113]:
# make a new "melted dataframe"


In [114]:
# preview melted dataframe


In [115]:
# now use this data to make aa line graph of changes over time


In [116]:
# view line graph


## Exporting Data 

Here, we'll export our crime, 911 call, and arrest datasets since we've cleaned them a bit (to use in the future)

In [117]:
# export crime as a csv


In [118]:
# export 911 call as a csv


In [119]:
# export arrest as a csv
